In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
df_genres = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_dummies.parquet')
df_games = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_games_clean.parquet')

In [4]:
df_merged = df_games.merge(df_genres, on='id', how='left')

features = ['release_date'] + list(df_genres.columns[1:])  

# Normalize 'release_year'
scaler = StandardScaler()
df_merged['release_date'] = scaler.fit_transform(df_merged[['release_date']])


df_final = df_merged[['id'] + features]
df_final= df_final.merge(df_games[['id', 'app_name']], on='id', how='left')

In [22]:
# Sample a subset of your data, for example, 50%
df_sampled = df_final.sample(frac=0.2, random_state=42)

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
similarity_matrix = np.nan_to_num(similarity_matrix)

In [23]:
def recomendacion_juego_muestreado(game_id, top_n=5):
    # Obtener los IDs de los juegos muestreados
    ids_juegos_muestreados = df_sampled['id'].unique()
    
    # Verificar si el ID del juego está en los juegos muestreados
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    # Obtener el índice del juego en los datos muestreados
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    # Verificar si se encontró el juego en los datos muestreados
    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]

    # Calcular los puntajes de similitud entre juegos
    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos similares
    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    
    # Obtener los nombres de los juegos similares
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

    # Mensaje de recomendación
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares


In [24]:
test_game_id = "774276"
recommendations = recomendacion_juego_muestreado(test_game_id, top_n=5)

print("Recomendaciones para ID de juego", test_game_id)
for rec in recommendations:
    print(rec)

Recomendaciones para ID de juego 774276
Juegos recomendados basados en el ID del juego 774276 - Imhotep, Pyramid Builder:
TS Marketplace: Feather River Canyon Scenario Pack 01
FSX: Steam Edition - Grumman HU-16B Albatross Add-On
TANE DLC: Newcastle Shunter
FSX Steam Edition: Endelave Airport Add-On
FSX: Steam Edition - Nordborg Airfield Add-On
